# 练习6：动态构建系统
-------
>本节练习节选自书籍《500 lines or less》——Contingent: A Fully Dynamic Build System

## 介绍

构建系统（build system）用于将源代码生成用户可用的目标（如库、可执行文件、脚本等），常见的有 GNU Make、CMake、Apache Ant 等。Python 中的 PyInstaller 也是构建系统的一种。本练习中，我们将实现一个构建系统，且试图对“动态交叉引用”问题提出一个解决方案。

长期以来，构建系统一直是计算机编程中的标准工具。

标准make构建系统的作者赢得了ACM软件系统奖，该标准构建系统于1976年首次开发。它不仅可以让您声明输出文件取决于一个（或多个）输入，还可以递归地进行操作。例如，程序可能取决于目标文件，而目标文件本身取决于相应的源代码：
```shell
    prog: main.o
            cc -o prog main.o

    main.o: main.c
            cc -C -o main.o main.c
```            
如果make在下次调用时发现main.c源代码文件的修改时间比main.o的更新时间更长，那么它不仅会重建main.o对象文件，而且还会重建它。 也将自己重建prog。

构建系统是分配给本科计算机科学专业学生的一个普通的学期项目，这不仅是因为构建系统几乎用在所有软件项目中，而且因为构建系统涉及基本数据结构和涉及有向图的算法（本章将在后面详细讨论） ）。

在构建系统背后经过数十年的使用和实践之后，人们可能会希望它们已完全成为通用的系统，甚至可以满足最奢侈的需求。

但是，实际上，构建构件之间的一种常见交互作用（动态交叉引用问题）在大多数构建系统中都处理得很差，以至于在本章中，我们受到启发，不仅要练习经典的解决方案和用于解决问题的数据结构，的make问题，而是要显着延长该解决方案，以一个更为苛刻的领域。

问题又是交叉引用。交叉引用会在哪里出现？在文本文档，文档和印刷书籍中！

### 1 问题：构建文档系统

从源重建格式化文档的系统似乎总是做太多或做很少的工作。

当他们响应较小的编辑时，使您等待不相关的章节被重新解析和重新设置格式时，它们会执行过多的工作。但是它们也可能重建得很少，从而给您带来不一致的最终产品。

考虑一下Sphinx，它是用于正式Python语言文档和Python社区中许多其他项目的文档构建器。Sphinx项目的`index.rst`内容通常包括一个目录：

```
   Table of Contents
   =================

   .. toctree::

      install.rst
      tutorial.rst
      api.rst
```

该章节文件名列表告诉Sphinx在构建`index.html`输出文件时，包括指向三个命名章节中每个章节的链接。它还将包含指向每一章中任何部分的链接。除去其标记，上述标题和`toctree`命令产生的文本可能是：

```
  Table of Contents

  • Installation

  • Newcomers Tutorial
      • Hello, World
      • Adding Logging

  • API Reference
      • Handy Functions
      • Obscure Classes
      `
```
如您所见，此目录是来自四个不同文件的信息的汇总。它的基本顺序和结构来自`index.rst`，而每章和节的实际标题均从这三章源文件本身中提取。

如果您以后重新考虑本教程的章节标题，那么您将编辑第一行`tutorial.rst` 并写点更好的东西：
```
  -Newcomers Tutorial
  +Beginners Tutorial
   ==================

   Welcome to the tutorial!
   This text will take you through the basics of...
```

当您准备重建时，Sphinx会做正确的事！它将重新构建教程章节本身和索引。（将输出管道输入到`cat`中使Sphinx成为在单独的行中宣布每个重建的文件，而不是使用空回车，用这些进度更新重复覆盖一行。）

```
   $ make html | cat
   writing output... [ 50%] index
   writing output... [100%] tutorial
```   

因为Sphinx选择重建两个文档，所以`tutorial.html`现在不仅将其新标题放在顶部，而且输出`index.html`还将在目录中显示更新的章节标题。 Sphinx重建了所有内容，以使输出保持一致。

如果对`tutorial.rst`的编辑较小，该怎么办？

```
   Beginners Tutorial
   ==================

  -Welcome to the tutorial!
  +Welcome to our project tutorial!
   This text will take you through the basics of...
```
在这种情况下，无需重建`index.html`，因为对段落内部进行的较小编辑不会更改目录中的任何信息。

但是事实证明，Sphinx并不像刚出现时那样聪明！

即使结果完全一样，它将继续执行重建`index.html`的多余工作。

```
   writing output... [ 50%] index
   writing output... [100%] tutorial
```

您可以在`index.html`的“之前”和“之后”版本上运行“ diff”，以确认您的小修改对首页没有影响-但是Sphinx还是让您等待它的重建。

您甚至可能没有注意到对于易于编译的小型文档的额外重建工作。但是，当您频繁调整和编辑冗长，复杂的文档或涉及诸如绘图或动画之类的多媒体生成的文档时，对工作流程的延迟会变得非常重要。

尽管Sphinx至少会在不做任何更改的情况下努力不重建每一章-例如，它并没有响应`“ tutorial.rst”`编辑而重建`install.html`或`api.html`， 它所做的超出了必要。

但是事实证明，Sphinx的作用甚至更糟：有时它做得太少，使您看到的输出不一致，用户可能会注意到。

要查看其最简单的故障之一，请首先在您的API文档的顶部添加一个交叉引用：

```
   API Reference
   =============

  +Before reading this, try reading our :doc:`tutorial`!
  +
   The sections below list every function
   and every single class and method offered...
```

对于目录，Sphinx通常会谨慎行事，将尽职地重建此API参考文档以及项目的`index.html`主页：


```
   writing output... [ 50%] api
   writing output... [100%] index
```

在`api.html`输出文件中，您可以确认Sphinx是否已将标题包含在交叉引用的定位标记中：

```html
   <p>Before reading this, try reading our
   <a class="reference internal" href="tutorial.html">
     <em>Beginners Tutorial</em>
   </a>!</p>
```

如果您现在再次对“ tutorial.rst”文件顶部的标题进行编辑怎么办？

您将使*三个*输出文件无效：

1.现在`tutorial.html`顶部的标题已过期，因此需要重建文件。

2. `index.html`中的目录仍然具有旧标题，因此需要重建文档。

3.`api.html`第一段中的嵌入式交叉引用仍然具有旧的章节标题，因此也需要重新构建。

Sphinx会做什么呢？
```
   writing output... [ 50%] index
   writing output... [100%] tutorial
```
哎呀

仅重建了两个文件，而不是三个。

Sphinx无法正确重建您的文档。

如果您现在将`HTML`推送到网络上，则用户将在`api.html`顶部的交叉引用中看到旧标题，但是一旦链接将其带到`tutorial.html`，用户将看到另一个标题（新标题）。 `本身。

Sphinx支持的多种交叉引用可能会发生这种情况：章标题，节标题，段落，类，方法和函数。

## 2 构建系统和一致性

上面概述的问题并非特定于Sphinx。它不仅困扰着其他文档系统（例如LaTeX），而且甚至会困扰那些只是试图用古老的make工具指导编译步骤的项目，如果它们的资产碰巧以交叉方式进行了交叉引用。

由于该问题是古老且普遍存在的，因此其解决方案具有同样长的沿袭：

```bash
   $ rm -r _build /
   $ make html
```
如果删除所有输出，则可以保证完全重建！有些项目甚至将别名`rm -r`命名为一个目标，`clean`因此只需快速`make clean`擦拭即可。

通过消除每一个中间或输出资产的每个副本，一个庞大的`rm -r`团队能够迫使该构建重新开始，而不会缓存任何内容，而不会存储可能会导致产品过时的早期状态。

但是我们可以开发出更好的方法吗？

如果您的构建系统是一个持续的过程，当它从一个文档的源代码传递到另一个文档的文本时，注意到每个章节标题，每个章节标题和每个交叉引用的短语，该怎么办？它关于更改单个源文件后是否重建其他文档的决定可以是精确的，而不是仅仅猜测，而是可以纠正的，而不是使输出保持不一致状态。

结果将是一个像旧的静态`make`工具一样的系统，但是该系统在构建文件时就了解了文件之间的依赖关系-在添加，更新和删除交叉引用时动态地添加和删除了依赖关系。

在以下各节中，我们将**使用Python构造一个名为Contingent的工具**。

Contingent在存在动态依赖项的情况下保证正确性，同时执行最少的重建步骤。尽管它可以应用于任何问题领域，但我们将针对上面概述的一小部分问题运行它。

## 3 链接任务以制作图形
任何构建系统都需要一种链接输入和输出的方法。例如，在我们上面的讨论中，三个标记文本分别产生一个相应的HTML输出文件。表达这些关系的最自然的方法是将它们组合成一个盒子和箭头（或者用数学术语来说是节点和边缘）来形成图形。

![](figure1.png) 

通过解析三个输入文本生成的三个文件。

程序员用来解决构建系统问题的每种语言都将提供各种数据结构，用这些数据结构可以表示节点和边的图形。

我们如何用Python表示这样的图？

Python语言通过直接支持四种通用数据结构的语言语法来赋予它们优先级。您可以通过简单地在源代码中键入它们的文字表示形式来创建这些四大数据结构的新实例，并且它们的四个类型对象可以作为内置符号使用，而无需导入即可使用。

该元组是用于保存异构数据只读序列-在元组中的每个时隙典型地是指不同的东西。在这里，元组将主机名和端口号放在一起，如果重新排序元素，它将失去其含义：

```python
('dropbox.com', 443)
```

**list**是用于保存同质数据的可变序列-每个项目通常具有与对等项目相同的结构和含义。

列表既可以用于保留数据的原始输入顺序，也可以重新排列或排序以建立新的更有用的顺序。

```python
['C', 'Awk', 'TCL', 'Python', 'JavaScript']
```

**set**不保留顺序。 集合仅记住是否已添加给定值，而不记住多少次，因此记住用于从数据流中删除重复项的数据结构。 例如，以下两个集合将各自包含三个元素：

```python
{3, 4, 5}
{3, 4, 5, 4, 4, 3, 5, 4, 5, 3, 4, 5}
```

**dict**是用于存储键可访问值的关联数据结构。Dicts允许程序员选择索引每个值的键，而不是像tuple和list那样使用自动整数索引。查找由一个散列表支持，这意味着无论dict有12个键还是有100万个键，查找dict键的速度都是相同的。


```python
{'ssh': 22, 'telnet': 23, 'domain': 53, 'http': 80}
```

Python灵活性的关键在于这四个数据结构是可组合的。 程序员可以将它们彼此任意嵌套以产生更复杂的数据存储，其规则和语法仍然是基本元组，列表，集合和字典中的简单规则。

假设我们的每个图形边缘都需要至少知道其原始节点和目标节点，那么最简单的表示可能就是元组。

顶部可能看起来像：

```python
    ('tutorial.rst', 'tutorial.html')
```

我们如何存储多个边缘？ 虽然我们最初的冲动可能只是简单地将所有边缘元组放入列表中，但这会带来不利条件。 列表会谨慎地保持顺序，但是谈论图形中边的绝对顺序没有意义。 即使我们只希望能够在`tutorial.rst`和` tutorial.html`之间绘制单个箭头，列表也会非常乐意保存完全相同的边缘的多个副本。 因此，正确的选择是集合，这将使我们表示为：

```python
    {('tutorial.rst', 'tutorial.html'),
     ('index.rst', 'index.html'),
     ('api.rst', 'api.html')}
```

这将允许我们所有边缘的快速迭代，单个边缘的快速插入和删除操作，以及一种检查特定边缘是否存在的快速方法。

不幸的是，这些并不是我们唯一需要的操作。

像Contingent这样的构建系统需要了解给定节点与连接到该节点的所有节点之间的关系。 例如，当`api.rst`更改时，Contingent需要知道哪些资产（如果有）受该更改影响，以最大程度地减少执行的工作并确保完整的构建。 要回答这个问题-`api.rst`下游有哪些节点？” —我们需要检查`api.rst`中的“出局”边缘。

但是构建依赖关系图需要Contingent也要考虑节点的`inputs`。 例如，当构建系统组装输出文档`tutorial.html`时，使用了哪些输入？ 通过观察每个节点的输入，Contingent可以知道`api.html`依赖于`api.rst`，而`tutorial.html`则不依赖。

当源发生更改并进行重建时，Contingent会重建每个更改的节点的传入边缘，以删除潜在的陈旧边缘，并重新学习任务这次使用的资源。

我们的元组组很难回答这些问题中的任何一个。 如果我们需要了解`api.html`与图的其余部分之间的关系，则需要遍历整个集合以查找以`api.html`节点开头或结尾的边。

像Python的`dict`这样的关联数据结构将允许直接从特定节点中查找所有边缘，从而使这些琐事变得更加容易：
```python
    {'tutorial.rst': {('tutorial.rst', 'tutorial.html')},
     'tutorial.html': {('tutorial.rst', 'tutorial.html')},
     'index.rst': {('index.rst', 'index.html')},
     'index.html': {('index.rst', 'index.html')},
     'api.rst': {('api.rst', 'api.html')},
     'api.html': {('api.rst', 'api.html')}}
```
查找特定节点的边缘现在将非常快，其代价是必须将每个边缘存储两次：一次存储在一组传入边缘中，一次存储在一组向外边缘中。

但是必须手动检查每组中的边缘，以查看哪些入站和哪些出站。 在节点的边缘集中不断重复命名节点也是有点多余的。

这两个反对意见的解决方案是将传入和传出的边放置在它们自己单独的数据结构中，这也将使我们不必为涉及的每个边都一遍又一遍地提及该节点。

```python
    incoming = {
        'tutorial.html': {'tutorial.rst'},
        'index.html': {'index.rst'},
        'api.html': {'api.rst'},
        }

    outgoing = {
        'tutorial.rst': {'tutorial.html'},
        'index.rst': {'index.html'},
        'api.rst': {'api.html'},
        }
```
注意，“ outgoing”直接用Python语法表示了我们之前所写的内容：构建系统会将左侧的源文档转换为右侧的输出文档。

对于这个简单的示例，每个源仅指向一个输出-所有输出集都只有一个元素-但是不久之后我们将看到示例，其中单个输入节点具有多个下游后果。

该集合字典数据结构中的每个边都得到两次表示，一次是从一个节点的出站边缘（` tutorial.rst`→` tutorial.html`），另一次是到另一节点的传入边缘（`tutorial.html`←`tutorial.rst`）。

只是从边缘任一端的两个节点的相反角度来看，这两种表示形式捕获了完全相同的关系。

但是作为这种冗余的回报，数据结构支持Contingent需要的快速查找。

## 4 Class的使用

您可能对以上关于Python数据结构的讨论中缺少类感到惊讶。毕竟，类是构建应用程序的一种常见机制，并且在其拥护者和批评者之间进行激烈辩论的频率并不高。曾经有人认为班级很重要，可以围绕它们设计整个教育课程，并且大多数流行的编程语言都包含用于定义和使用它们的专用语法。

但是事实证明，类通常与数据结构设计问题正交。类没有为我们提供完全替代的数据建模范例，而是仅重复了我们已经看到的数据结构：

- 类实例被*实现*为字典。
- 类实例的*使用*就像可变的元组。
该类通过更漂亮的语法提供键查找，您可以在其中用`graph.incoming`代替`graph["incoming"]`。但是，实际上，类实例几乎从未用作通用键值存储。相反，它们用于按属性名称组织相关但异构的数据，实现细节封装在一致且令人难忘的接口后面。

因此，您不必创建一个主机名和一个端口号在元组中，而是必须记住哪个名在前，哪个名在后，而创建一个`Address`类，其实例分别具有`host`和`port`属性。然后，您可以将`Address`对象传递到否则会有匿名元组的位置。代码变得更易于阅读和编写。但是，使用类实例并不能真正改变我们在进行数据设计时遇到的任何问题。它只是提供了一个更漂亮，更匿名的容器。

因此，类的真正价值不是在于它们改变了数据设计的科学。类的价值在于它们使您可以从程序的其余部分隐藏数据设计！

成功的应用程序设计取决于我们利用Python提供的强大的内置数据结构的能力，同时最大程度地减少了随时需要记住的细节量。类提供了解决这一明显难题的机制：有效使用类，可以围绕系统整体设计的一些小子集提供外观。在一个子集（`Graph`例如`a`）中工作时，只要记住其他子集的接口，我们就可以忘记其他实现的细节。这样，程序员通常会发现自己在编写系统的过程中处于多个抽象层次之间，现在正在使用特定子系统的特定数据模型和实现细节，现在通过其接口连接了较高层次的概念。

例如，从外部，代码可以简单地请求一个新`Graph`实例：

In [1]:
from contingent import graphlib
g = graphlib.Graph()

无需了解Graph工作原理的详细信息。仅使用图形的代码在处理图形时（例如添加边或执行其他一些操作时）仅看到接口动词（即方法调用）：

In [2]:
g.add_edge('index.rst', 'index.html')
g.add_edge('tutorial.rst', 'tutorial.html')
g.add_edge('api.rst', 'api.html')

在没有显式创建“ node”和“ edge”对象的情况下，我们在图形中添加了边，并且在这些早期示例中，节点本身只是字符串。

Python语言和社区明确并有目的地强调使用简单的通用数据结构来解决问题，而不是为要解决的问题的每一个细节创建自定义类。这是`Pythonic`解决方案概念的一个方面：Pythonic解决方案试图最大程度地减少语法开销，并利用Python强大的内置工具和广泛的标准库。

考虑到这些考虑因素，让我们回到`Graph`类，检查其设计和实现，以查看数据结构和类接口之间的相互作用。

Graph构造新实例时，已经使用上一节中概述的逻辑构建了一对字典来存储边：

```python
class Graph:
    """A directed graph of the relationships among build tasks."""

    def __init__(self):
        self._inputs_of = defaultdict(set)
        self._consequences_of = defaultdict(set)
```

在属性名称前面的前导下划线`_inputs_of`和`_consequences_of `是在Python社区信号共同约定的属性是私有的。这种约定是社区建议程序员通过空间和时间彼此传递消息和警告的一种方式。认识到需要指出公共对象属性和内部对象属性之间的差异，社区采用了单个前导下划线作为对其他程序员（包括我们将来的自己）的简洁一致的指示，即该属性最好被视为内部无形内部机制的一部分。班级。

为什么我们使用`defaultdict`标准指令而不是标准指令？将字典与其他数据结构组成时的常见问题是处理缺少的键。在正常情况下，检索不存在的键将引发`KeyError`：

In [3]:
consequences_of = {}
consequences_of['index.rst'].add('index.html')

KeyError: 'index.rst'

这种需求非常普遍，以至于Python包含一个特殊的实用工具，`defaultdict`您可以通过它提供一个返回缺少键值的函数。当我们询问`Graph`尚未看到的边缘时，我们将得到一个空的`set`而不是一个异常：

In [4]:
from collections import defaultdict
consequences_of = defaultdict(set)
consequences_of['api.rst']

set()

通过这种方式来构造我们的实现，意味着每个键的首次使用看上去都与使用特定键的第二次及以后相同：

In [5]:
consequences_of['index.rst'].add('index.html')
'index.html' in consequences_of['index.rst']

True

有了这些技术，我们就可以检查的实现`add_edge`，我们之前曾使用它来构建图形。

```python
    def add_edge(self, input_task, consequence_task):
        """Add an edge: `consequence_task` uses the output of `input_task`."""
        self._consequences_of[input_task].add(consequence_task)
        self._inputs_of[consequence_task].add(input_task)
```
这种方法掩盖了以下事实：每个新边都需要两个（而不是一个）存储步骤，以便我们在两个方向上都知道。并注意如何`add_edge()`不知道或不在乎之前是否曾见过任何一个节点。由于输入和后果数据结构均为`a defaultdict(set)`，因此该`add_edge()`方法对于节点的新颖性仍然一无所知- `defaultdict`通过动态创建新`set`对象来解决差异。正如我们在上面看到的，如果不使用`defaultdict`，`add_edge()`时间将增加三倍。更重要的是，对结果代码的理解和推理将更加困难。此实现演示了`Pythonic`解决问题的方法：简单，直接和简洁。

还应该为调用者提供一种访问每个边缘的简单方法，而不必学习如何遍历我们的数据结构：
```python
    def edges(self):
        """Return all edges as ``(input_task, consequence_task)`` tuples."""
        return [(a, b) for a in self.sorted(self._consequences_of)
                       for b in self.sorted(self._consequences_of[a])]
```

该`Graph.sorted()`方法尝试按照可以为用户提供稳定输出顺序的自然排序顺序（例如字母顺序）对节点进行排序。

通过使用这种遍历方法，我们可以看到，在前面的三个` add`方法调用之后， `g`现在表示如下：

In [6]:
from pprint import pprint
pprint(g.edges())

[('api.rst', 'api.html'),
 ('index.rst', 'index.html'),
 ('tutorial.rst', 'tutorial.html')]


由于我们现在有了一个真实的实时Python对象，而不仅仅是一个图形，因此我们可以向它提出有趣的问题！例如，当Contingent从源文件构建博客时，它将需要知道诸如“什么取决于`api.rst`？”之类的内容。当`api.rst`内容更改时：

In [7]:
g.immediate_consequences_of('api.rst')

['api.html']

`g`告诉Contingent，当`api.rst`更改时，`api.html`就会过时，必须重新构建。

`index.html`呢？

In [8]:
g.immediate_consequences_of('index.html')

[]

返回了一个空列表，表示`index.html`在图的右边缘，因此如果更改，则无需再构建任何东西。由于已经进行了布局数据的工作，因此可以非常简单地表示此查询：
```python
    def immediate_consequences_of(self, task):
        """Return the tasks that use `task` as an input."""
        return self.sorted(self._consequences_of[task])
```


In [10]:
from contingent.rendering import as_graphviz
open('figure1.dot', 'w').write(as_graphviz(g)) and None

我们将为每个需要通过解析输入文件生成然后传递给我们的其他例程之一的标题字符串创建一个节点：

In [11]:
g.add_edge('api.rst', 'api-title')
g.add_edge('api-title', 'index.html')
g.add_edge('tutorial.rst', 'tutorial-title')
g.add_edge('tutorial-title', 'index.html')

In [12]:
pprint(g.edges())

[('api-title', 'index.html'),
 ('api.rst', 'api-title'),
 ('api.rst', 'api.html'),
 ('index.rst', 'index.html'),
 ('tutorial-title', 'index.html'),
 ('tutorial.rst', 'tutorial-title'),
 ('tutorial.rst', 'tutorial.html')]


![](figure2.png)

只要提及的标题发生变化，`index.html`随时准备重建。
本手册演练说明了Contingent最终将为我们做些什么：该图`g`捕获了项目文档中各种工件的输入和后果。

## 5 学习联系
现在，我们有了一种方法，让Contingent可以跟踪任务及其之间的关系。但是，如果我们更仔细地查看上图，我们会发现它实际上有点波折和模糊：`api.rst`是怎么产生`api.html`的？我们如何知道`index.html`需要教程中的标题？以及如何解决这种依赖性？

当我们手动构建后果图时，我们对这些想法的直觉概念就起作用了，但是不幸的是，计算机并不是非常直观的，因此我们需要更精确地了解我们想要的东西。

从源产生输出需要采取什么步骤？如何定义和执行这些步骤？Contingent如何知道它们之间的联系？

在Contingent中，构建任务被定义为“函数加参数”。

- 这些函数定义特定项目理解如何执行的动作。
- 这些参数提供了具体信息：应阅读哪个源文档，需要哪个博客标题。

当它们运行时，这些函数可以依次调用其他任务函数，并传递它们需要答案的任何参数。

为了了解它是如何工作的，我们现在实际上将实现开头描述的文档构建器。为了避免陷入困境，在本例中，我们将使用简化的输入和输出文档格式。我们的输入文档将在第一行包含一个标题，其余文本构成正文。交叉引用将只是反引号中包含的源文件名，在输出中将其替换为输出中相应文档的标题。

以下是示例`index.txt`，`api.txt`和`tutorial.txt`的内容，包括格式的标题，文档正文和交叉引用：

In [14]:
index = """
    Table of Contents
    -----------------
    * `tutorial.txt`
    * `api.txt`
    """

tutorial = """
    Beginners Tutorial
    ------------------
    Welcome to the tutorial!
    We hope you enjoy it.
    """

api = """
    API Reference
    -------------
    You might want to read
    the `tutorial.txt` first.
    """

现在我们有了一些可以使用的原始资料，基于Contingent的博客构建者需要哪些功能？

在上面的简单示例中，HTML输出文件直接从源代码开始，但是在实际的系统中，将源代码转换为标记涉及几个步骤：
- 从磁盘读取原始文本
- 将文本解析为方便的内部表示形式
- 处理所有指令。

作者可能已经指定，解决了交叉引用或其他外部依赖项（例如include文件），并应用了一个或多个视图转换将内部表示形式转换为其输出形式。

Contingent通过将任务分组到一个“Project”来管理任务，这是一种构建系统的多管闲事者，它将自己注入到构建过程的中间，注意到每次一个任务与另一个任务对话，以构建所有任务之间的关系图。


In [15]:
from contingent.projectlib import Project, Task
project = Project()
task = project.task


本练习开头给出的示例的构建系统可能涉及一些任务。

我们的`read()`任务将假装从磁盘读取文件。 由于我们确实在变量中定义了源文本，因此只需将文件名转换为相应的文本即可。

In [17]:
filesystem = {'index.txt': index,
              'tutorial.txt': tutorial,
              'api.txt': api}
@task
def read(filename):
    return filesystem[filename]

`parse()`任务根据我们文档格式的规范解释文件内容的原始文本。

我们的格式非常简单：文档标题显示在第一行，其余内容被视为文档正文。

In [18]:
@task
def parse(filename):
    lines = read(filename).strip().splitlines()
    title = lines[0]
    body = '\n'.join(lines[2:])
    return title, body

请注意`parse()`和`read()`之间的解析的第一个任务是将给定的文件名传递给`read()`，文件名将查找并返回该文件的内容。

`title_of()`给定源文件名称的任务将返回文档的标题：

In [20]:
@task
def title_of(filename):
    title, body = parse(filename)
    return title

最后的任务， `render()`将文档的内存表示形式转换为输出形式。实际上，它是`parse()`的倒数。而`parse()`采用符合规范的输入文档并将其转换为内存中表示， `render()`采用内存中表示并生成符合某些规范的输出文档。

In [19]:
import re
LINK = '<a href="{}">{}</a>'
PAGE = '<h1>{}</h1>\n<p>\n{}\n<p>'

def make_link(match):
    filename = match.group(1)
    return LINK.format(filename, title_of(filename))

@task
def render(filename):
    title, body = parse(filename)
    body = re.sub(r'`([^`]+)`', make_link, body)
    return PAGE.format(title, body)

In [21]:
print(render('tutorial.txt'))

<h1>Beginners Tutorial</h1>
<p>
    Welcome to the tutorial!
    We hope you enjoy it.
<p>


![](figure3.png)
说明任务图，该任务图可过渡地连接生成输出所需的所有任务，从读取输入文件到解析和转换文档并呈现文档。

每次调用新任务时，Contingent都可以假定当前位于堆栈顶部的任务已调用该任务，并且将使用其输出。维护堆栈将需要几个额外的步骤来围绕任务$T$的调用：

- 将$T$推入堆栈。
- 执行$T$，让它调用它需要的任何其他任务。
- 将$T$弹出堆栈。
- 返回其结果。

为了拦截任务调用，Project利用了Python的一项关键功能：`function decorators`。装饰器可以在定义函数时对其进行处理或转换。该`Project.task`装饰用这个机会来包装的另一个功能，里面每个任务的包装，这使得包装之间的责任完全分离-这有可能会担心图形和堆栈管理代表项目-而我们的任务功能专注于文档处理。

这是`task`装饰器样板的外观：

```python
        from functools import wraps

        def task(function):
            @wraps(function)
            def wrapper(*args):
                # wrapper body, that will call function()
            return wrapper
```
这是一个典型的Python装饰器声明。然后，可以通过`@`在`def`创建函数的字符顶部命名该函数，将其应用于函数：

```python
    @task
    def title_of(filename):
        title, body = parse(filename)
        return title
```
完成此定义后，`title_of`将引用该函数的包装版本。包装器可以通过名称访问函数的原始版本`function`，并在适当的时间对其进行调用。Contingent包装器的主体运行如下内容：

```python
    def task(function):
        @wraps(function)
        def wrapper(*args):
            #----------------
            task = Task(wrapper, args)
            if self.task_stack:
                self._graph.add_edge(task, self.task_stack[-1])
            self._graph.clear_inputs_of(task)
            self._task_stack.append(task)
            try:
                value = function(*args)
            finally:
                self._task_stack.pop()

            return value
            #---------------
        return wrapper
```

该包装器执行几个关键的维护步骤：

1. 为方便起见，将任务（一个函数及其参数）打包到一个小对象中。wrapper在此命名为函数的包装版本。

2. 如果此任务已由正在执行的当前任务调用，则添加一条边，以捕获该任务是已在运行的任务的输入这一事实。

3. 忘记我们上一次可能在该任务上学到的知识，因为这一次可能会做出新的决定-例如，如果API指南的源文本不再提及该Tutorial，那么`render()`它将不再请求该`Tutorial`文档`title_of()`。

4. 将该任务推入任务堆栈的顶部，以防其决定在执行工作时调用其他任务。

5. 在`try...finally`块内调用任务，以确保我们正确完成了从堆栈中删除的任务，即使该任务因引发异常而死亡。

6. 返回任务的返回值，以便此包装的调用者将无法得知他们没有简单地调用普通任务函数本身。

步骤4和5维护任务堆栈本身，然后由步骤2用于执行结果跟踪，这是我们首先构建任务堆栈的全部原因。

由于每个任务都被其自身的包装函数副本所包围，因此，正常任务堆栈的单纯调用和执行将产生关系图，这是看不见的副作用。因此，我们谨慎地在定义的每个处理步骤周围使用包装器：
```python
    @task
    def read(filename):
        # body of read

    @task
    def parse(filename):
        # body of parse

    @task
    def title_of(filename):
        # body of title_of

    @task
    def render(filename):
        # body of render
```

当我们调用`parse('tutorial.txt') `装饰器时，我们了解了`parse`和`read`之间的联系。我们可以通过建立另一个`Task`元组来询问这种关系，并询问如果其输出值更改会带来什么后果：

In [22]:
task = Task(read, ('tutorial.txt',))
print(task)

read('tutorial.txt')


In [23]:
project._graph.immediate_consequences_of(task)

[parse('tutorial.txt')]

重新读取`tutorial.txt`文件并发现其内容已更改的结果时，我们需要重新执行该文档的`parse()`例程。

如果我们渲染整个文档集会怎样？Contingent是否能够学习整个构建过程？

In [24]:
for filename in 'index.txt','tutorial.txt','api.txt':
    print(render(filename))
    print('=' * 30)

<h1>Table of Contents</h1>
<p>
    * <a href="tutorial.txt">Beginners Tutorial</a>
    * <a href="api.txt">API Reference</a>
<p>
<h1>Beginners Tutorial</h1>
<p>
    Welcome to the tutorial!
    We hope you enjoy it.
<p>
<h1>API Reference</h1>
<p>
    You might want to read
    the <a href="tutorial.txt">Beginners Tutorial</a> first.
<p>


It works！

从输出中，我们可以看到转换将源文档中的指令标题替换为文档标题，表明Contingent能够发现构建文档所需的各种任务之间的联系。

![](figure4.png)

通过观察一个任务，通过task包装机调用另一个任务， Project就自动了解了输入和后果图。由于它具有完整的结果图可供使用，如果任何任务的输入发生变化，Contingent都知道要重建的所有事物。

## 6 追踪后果

初始构建运行完成后，Contingent需要监视输入文件的更改。当用户完成一个新的编辑并运行“保存”时，该`read()`方法及其*后果*都需要被调用。

这将要求我们以与创建图形相反的顺序移动图形。您会回想起，它是通过为API参考调用`render()`和`parse()`，并最终调用该`read()`任务而构建的。现在我们朝另一个方向前进：我们知道`read()`现在将返回新的内容，并且我们需要弄清楚其将产生什么后果。

编译结果的过程是一个递归过程，因为每个结果本身可以有其他依赖于此的任务。我们可以通过重复调用图形来手动执行此递归。（请注意，我们在这里利用了Python提示符保存名称下显示的最后一个值_供后续表达式使用的事实。）

In [25]:
task = Task(read, ('api.txt',))
project._graph.immediate_consequences_of(task)

[parse('api.txt')]

In [26]:
t1, = _
project._graph.immediate_consequences_of(t1)

[render('api.txt'), title_of('api.txt')]

In [27]:
t2, t3 = _
project._graph.immediate_consequences_of(t2)

[]

In [28]:
project._graph.immediate_consequences_of(t3)

[render('index.txt')]

In [30]:
t4, = _
project._graph.immediate_consequences_of(t4)

[]

这种递归任务不断地寻找直接的结果，只有当我们到达没有进一步结果的任务时才停止，这是一种足够基本的图形操作，`Graph`类中的一个方法直接支持它：

In [31]:
# Secretly adjust pprint to a narrower-than-usual width:
_pprint = pprint
pprint = lambda x: _pprint(x, width=40)
pprint(project._graph.recursive_consequences_of([task]))

[parse('api.txt'),
 render('api.txt'),
 title_of('api.txt'),
 render('index.txt')]


实际上，`recursive_consequences_of()尝`试变得聪明一点。如果某个特定任务由于其他多个任务的下游结果而重复出现，则应注意在输出列表中仅提及一次，并将其移至末尾，以便仅在作为其输入的任务之后出现。这种智能由拓扑排序的经典深度优先实现实现，该算法通过隐藏的递归辅助函数在Python中编写起来相当容易。查看[graphlib.py](contingent/graphlib.py)源代码以获取详细信息。

如果在检测到更改后，我们谨慎地重新运行递归结果中的每个任务，那么Contingent将能够避免重建得太少。但是，我们的第二个挑战是避免重建过多。再次参考下图。
![](figure4.png)

我们希望避免每次`tutorial.txt`更改时都重建所有三个文档，因为大多数编辑可能不会影响其标题，而只会影响其正文。如何做到这一点？

解决方案是使图形重新计算依赖于缓存。当逐步解决更改的递归结果时，我们将仅调用输入与上次不同的任务。

此优化将涉及最终的数据结构。我们将提供`Project`一个`_todo`集合，用于记住每个至少更改了一个输入值并因此需要重新执行的任务。因为只有`_todo`已过期的任务，所以构建过程可以跳过运行任何任务，除非它们出现在其中。

同样，Python方便且统一的设计使这些功能非常易于编码。由于任务对象是可散列的，因此 `_todo`可以简单地设置为一组集合，该集合可以通过标识记住任务项--保证任务永远不会出现两次--并且`_cache`先前运行的返回值可以是将任务作为键的命令。

更准确地说，只要`_todo`非空，重建步骤就必须保持循环。在每个循环中，它应该：

- 调用`recursive_consequences_of()`并传递中列出的每个任务`_todo`。返回值将不仅是`_todo`任务本身的列表，还包括任务下游的每个任务的列表-换句话说，如果这次输出不同，则可能需要重新执行每个任务。

- 对于列表中的每个任务，检查它是否在中列出`_todo`。如果没有，那么我们可以跳过运行它，因为在上游重新调用的所有任务都没有产生需要任务重新计算的新返回值。

- 但是，对于在`_todo`我们到达时确实列出的任何任务，我们需要要求它重新运行并重新计算其返回值。如果任务包装函数检测到此返回值与旧的缓存值不匹配，则`_todo`在我们将其下游任务返回到递归结果列表之前，它将自动添加到其下游任务。

当我们到达列表的末尾时，实际上可能需要重新运行的每个任务实际上应该已经重新运行。但以防万一，`_todo `如果尚未清空，我们将检查并重试。即使对于变化非常快的依赖树，这也应该很快解决。

只有一个循环（例如，任务A需要任务B的输出而任务 本身又需要任务A的输出）才可以使构建器处于无限循环中，并且前提是其返回值永远不会稳定。幸运的是，实际的构建任务通常没有周期。

让我们通过一个示例来跟踪该系统的行为。
```
tutorial = """
    Beginners Tutorial
    ------------------
    Welcome to the tutorial!
    We hope you enjoy it.
    """
```    
假设您编辑`tutorial.txt`，更改标题和正文内容。我们可以通过修改`filesystem dict`中的值来模拟这一点：

In [32]:
filesystem['tutorial.txt'] = """
    The Coder Tutorial
    ------------------
    This is a new and improved
    introductory paragraph.
    """

现在内容已更改，我们可以通过使用`cache_off()`上下文管理器要求项目重新运行该`read()`任务，该上下文管理器暂时禁用其对于给定任务和参数返回其旧缓存结果的意愿：

In [33]:
with project.cache_off():
...     text = read('tutorial.txt')

现在，新的教程文本已读入缓存。有多少下游任务需要重新执行？

为了帮助我们回答这个问题，Project该类支持一个简单的跟踪工具，该工具将告诉我们在重建过程中执行了哪些任务。由于上述更改`tutorial.txt` 影响到它的主体和标题，因此下游的所有内容都需要重新计算：

In [34]:
project.start_tracing()
project.rebuild()
print(project.stop_tracing())

calling parse('tutorial.txt')
calling render('tutorial.txt')
calling title_of('tutorial.txt')
calling render('api.txt')
calling render('index.txt')


![](figure4.png)
回顾上图，您会发现，正如预期的那样，这是`read('tutorial.txt')`的直接或下游结果中包含的所有任务。

但是，如果我们再次编辑它，但是这次标题保持不变怎么办？

In [35]:
filesystem['tutorial.txt'] = """
... The Coder Tutorial
... ------------------
... Welcome to the coder tutorial!
... It should be read top to bottom.
... """

with project.cache_off():
...     text = read('tutorial.txt')

In [36]:
project.start_tracing()
project.rebuild()
print(project.stop_tracing())

calling parse('tutorial.txt')
calling render('tutorial.txt')
calling title_of('tutorial.txt')


成功！

仅重建了一个文档。在`title_of()`给定新输入文档的情况下，尽管返回了相同的值，但这意味着所有其他下游任务均不受更改的影响，因此不会被重新调用。

## 结论

在Python中对Contingent进行编程时，我们跳过了诸如`TaskArgumentand`、`CachedResult`和`ConsequenceList`的十几种可能的类的创建。相反，我们借鉴了Python解决通用数据结构通用问题的悠久传统，导致代码重复使用了核心数据结构元组，列表，集合和字典中的一小部分想法。

归功于严格的封装原则（仅允许`Graph`代码触摸图形的集合，并允许代码触摸Project项目的集合），如果`set`操作在项目的后续阶段返回错误，将永远不会产生歧义。错误发生时最内部执行方法的名称必然会将我们定向到错误所涉及的类和集合。`set`只要我们将常规下划线放在数据结构属性的前面，然后注意不要从类外部的代码中碰到它们，就不必为数据类型的每种可能的应用创建的子类。